In [6]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

%cd /content/drive/MyDrive/Colab\ Notebooks/MovieSentimentalAnalysis/

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/MovieSentimentalAnalysis


In [7]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 49.9 MB/s 


In [8]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings(action='ignore')
import nltk
#nltk.download('all') # 처음 실행 시 주석을 제거하고 실행해주세요!
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize
from konlpy.tag import Okt 

In [9]:
data = pd.read_csv('dataset/train.csv') # train 데이터 불러오기
data.head() # 데이터 확인하기

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [11]:
data.isna().sum() # 결측치 확인

id          0
document    0
label       0
dtype: int64

In [12]:
train, val = train_test_split(data)
train.reset_index(inplace=True) # 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화해주었습니다.
val.reset_index(inplace=True)

In [ ]:
text = "Hello, nice to meet you. What's your name? Have a nice day! See you soon." # 예시 문장을 정의합니다.
print('문장 토큰화 결과 ==>',sent_tokenize(text))
print('단어 토큰화 결과 ==>', word_tokenize(text))

문장 토큰화 결과 ==> ['Hello, nice to meet you.', "What's your name?", 'Have a nice day!', 'See you soon.']
단어 토큰화 결과 ==> ['Hello', ',', 'nice', 'to', 'meet', 'you', '.', 'What', "'s", 'your', 'name', '?', 'Have', 'a', 'nice', 'day', '!', 'See', 'you', 'soon', '.']


In [23]:
#학습 셋 전처리
data['preprocessed'] = data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9 ]","") # 한글과 공백을 제외하고 모두 제거

# 인스턴스 할당
okt = Okt()
spacing = Spacing()

spaced = []
for sentence in data['preprocessed']:
    sentence = spacing(sentence)
    spaced.append(sentence)
data['spaced'] = pd.DataFrame(spaced)

tokenized = [] # 데이터프레임의 한 컬럼으로 추가할 리스트
for sentence in data['spaced']: # 전처리된 리뷰들을 하나씩 꺼내옵니다
    tokens = okt.morphs(sentence, stem = True) # 형태소 분석 (stem = True로 설정해 어간 추출을 해주었습니다)
    tokenize = " ".join(tokens) # tokens라는 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가해줍니다
data["tokenized_stem"] = pd.DataFrame(tokenized) # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 컬럼명으로 추가해줍니다.


'\nmain_pos = [] # 데이터프레임의 새 컬럼이 될 리스트\nfor sentence in data[\'preprocessed\']: # 리뷰들을 하나씩 가져옵니다\n    pos = okt.pos(sentence) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다\n    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.\n    main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.\n    main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니다\ndata["main_pos"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.\n'

In [52]:
main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
for sentence in data['spaced']: # 리뷰들을 하나씩 가져옵니다
    pos = okt.pos(sentence) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] not in ("Josa", "Number", "Foreign", "Suffix")] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.
    main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니다
data["main_pos"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.


In [53]:
data.head()

,id,document,label,preprocessed,tokenized_stem,spaced,main_pos
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,영상 이나 음악 이 이 쁘 다 해도 미화 시키다 불륜 일 뿐,영상이나 음악이 이 쁘다 해도 미화시킨 불륜일뿐,영상 음악 이 쁘 해도 미화 시킨 불륜 일 뿐
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯,히치콕 이 이 영화 를 보다 분명 박수 를 치다 듯,히치콕이 이 영화를 봤다면 분명 박수를 쳤을 듯,히치콕 이 영화 봤다면 분명 박수 쳤을 듯
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1,괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요,괜찮다 음악 영화 가 또 나오다 요 따뜻하다 겨울 이 되다 것 같다,괜찮은 음악영화가 또 나왔군 요 따뜻한 겨울이 될 것 같아요,괜찮은 음악 영화 또 나왔군 요 따뜻한 겨울 될 것 같아요
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0,아무래도 20년도지난작품이라 지금보기는너무유치하다,아무래도 20년 도 지난 작품 이라 지금 보기 는 너무 유치하다,아무래도 20년도 지난 작품이라 지금 보기는 너무 유치하다,아무래도 지난 작품 지금 보기 너무 유치하다
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0,지금까지의 영화들이 그랬듯 이 영화역시 일본에 대한 미화는 여전하다,지금 까지의 영화 들 이 그렇다 이 영화 역시 일본 에 대한 미화 는 여전하다,지금까지의 영화들이 그랬듯 이 영화 역시 일본에 대한 미화는 여전하다,지금 영화 그랬듯 이 영화 역시 일본 대한 미화 여전하다


In [51]:
txt = '지금까지의 영화들이 그랬듯 이 영화 역시 일본에 대한 미화는 여전하다'
pos = okt.pos(txt)
print(pos)

[('지금', 'Noun'), ('까지의', 'Josa'), ('영화', 'Noun'), ('들', 'Suffix'), ('이', 'Josa'), ('그랬듯', 'Adjective'), ('이', 'Noun'), ('영화', 'Noun'), ('역시', 'Noun'), ('일본', 'Noun'), ('에', 'Josa'), ('대한', 'Noun'), ('미화', 'Noun'), ('는', 'Josa'), ('여전하다', 'Adjective')]


In [54]:
for max_features in range(100, 10000, 100):
#max_features = 7100
  vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=max_features)

  X = data['main_pos']
  y = np.array(data.label)

  train, val, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=2)

  X_train = vectorizer.fit_transform(train)
  X_eval = vectorizer.transform(val)
  lgs = LogisticRegression(class_weight='balanced')
  lgs.fit(X_train, y_train)
  predicted = lgs.predict(X_eval)
  print(f"max_features: {max_features}, Accuracy: {lgs.score(X_eval, y_eval)}" )

max_features: 100, Accuracy: 0.705
max_features: 200, Accuracy: 0.734
max_features: 300, Accuracy: 0.754
max_features: 400, Accuracy: 0.779
max_features: 500, Accuracy: 0.786
max_features: 600, Accuracy: 0.798
max_features: 700, Accuracy: 0.797
max_features: 800, Accuracy: 0.807
max_features: 900, Accuracy: 0.818
max_features: 1000, Accuracy: 0.819
max_features: 1100, Accuracy: 0.835
max_features: 1200, Accuracy: 0.831
max_features: 1300, Accuracy: 0.837
max_features: 1400, Accuracy: 0.841
max_features: 1500, Accuracy: 0.846
max_features: 1600, Accuracy: 0.853
max_features: 1700, Accuracy: 0.851
max_features: 1800, Accuracy: 0.854
max_features: 1900, Accuracy: 0.855
max_features: 2000, Accuracy: 0.856
max_features: 2100, Accuracy: 0.857
max_features: 2200, Accuracy: 0.858
max_features: 2300, Accuracy: 0.859
max_features: 2400, Accuracy: 0.861
max_features: 2500, Accuracy: 0.858
max_features: 2600, Accuracy: 0.861
max_features: 2700, Accuracy: 0.86
max_features: 2800, Accuracy: 0.86
max

In [33]:
#val = pd.DataFrame(val)
#val.loc[:,'label'] = predicted
#val.loc[:,'answer'] = y_eval

In [ ]:
out = val[val['label']!=val['answer']]
out.to_csv("wrong.csv", index = False)

In [37]:
y_eval = np.array(val.label)
print(f"Accuracy: {lgs.score(X_eval, y_eval)}" )

AttributeError: ignored

In [ ]:
txt = '최우식 참 연기 잘하는 배우구나 라는 걸 느끼게 해 준 영화	'
tokens = okt.morphs(sentence, stem = True)
pos = okt.pos(txt)
print(tokens)
print(pos)

['여', '주인공', '이', '전작', '에서는', '귀엽다', '여', '기선', '완전', '망하다', '실망', '이다']
[('최우식', 'Noun'), ('참', 'Verb'), ('연기', 'Noun'), ('잘', 'VerbPrefix'), ('하는', 'Verb'), ('배우구나', 'Verb'), ('라는', 'Josa'), ('걸', 'Noun'), ('느끼게', 'Verb'), ('해', 'Noun'), ('준', 'Noun'), ('영화', 'Noun'), ('\t', 'Foreign')]


In [56]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

for max_features in range(100, 10000, 100):
#max_features = 7100
  vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=max_features)

  X = data['main_pos']
  y = np.array(data.label)

  train, val, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=2)

  X_train = vectorizer.fit_transform(train)
  X_eval = vectorizer.transform(val)
  forest = RandomForestClassifier(n_estimators = 100, random_state = 2022)
  forest = forest.fit(X_train, y_train)

  predicted = forest.predict(X_eval)
  print(f"max_features: {max_features}, Accuracy: {forest.score(X_eval, y_eval)}" )

max_features: 100, Accuracy: 0.708
max_features: 200, Accuracy: 0.749
max_features: 300, Accuracy: 0.74
max_features: 400, Accuracy: 0.769
max_features: 500, Accuracy: 0.774
max_features: 600, Accuracy: 0.778
max_features: 700, Accuracy: 0.794
max_features: 800, Accuracy: 0.801
max_features: 900, Accuracy: 0.805
max_features: 1000, Accuracy: 0.805
max_features: 1100, Accuracy: 0.816
max_features: 1200, Accuracy: 0.822
max_features: 1300, Accuracy: 0.824
max_features: 1400, Accuracy: 0.825
max_features: 1500, Accuracy: 0.823
max_features: 1600, Accuracy: 0.836
max_features: 1700, Accuracy: 0.824
max_features: 1800, Accuracy: 0.821
max_features: 1900, Accuracy: 0.821
max_features: 2000, Accuracy: 0.824
max_features: 2100, Accuracy: 0.826
max_features: 2200, Accuracy: 0.83
max_features: 2300, Accuracy: 0.829
max_features: 2400, Accuracy: 0.842
max_features: 2500, Accuracy: 0.828
max_features: 2600, Accuracy: 0.839
max_features: 2700, Accuracy: 0.834
max_features: 2800, Accuracy: 0.83
max_

In [61]:
#Support Vector Machine
from sklearn import svm

for max_features in range(100, 10000, 100):
#max_features = 7100
  vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(1, 3), max_features=max_features)

  X = data['main_pos']
  y = np.array(data.label)

  train, val, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=2)

  X_train = vectorizer.fit_transform(train)
  X_eval = vectorizer.transform(val)
  clf = svm.SVC()
  clf = clf.fit(X_train, y_train)

  predicted = clf.predict(X_eval)
  print(f"max_features: {max_features}, Accuracy: {clf.score(X_eval, y_eval)}" )

max_features: 100, Accuracy: 0.73
max_features: 200, Accuracy: 0.754
max_features: 300, Accuracy: 0.778
max_features: 400, Accuracy: 0.791
max_features: 500, Accuracy: 0.805
max_features: 600, Accuracy: 0.81
max_features: 700, Accuracy: 0.817
max_features: 800, Accuracy: 0.826
max_features: 900, Accuracy: 0.833
max_features: 1000, Accuracy: 0.836
max_features: 1100, Accuracy: 0.841
max_features: 1200, Accuracy: 0.845
max_features: 1300, Accuracy: 0.85
max_features: 1400, Accuracy: 0.851
max_features: 1500, Accuracy: 0.848
max_features: 1600, Accuracy: 0.856
max_features: 1700, Accuracy: 0.861
max_features: 1800, Accuracy: 0.866
max_features: 1900, Accuracy: 0.866
max_features: 2000, Accuracy: 0.863
max_features: 2100, Accuracy: 0.861
max_features: 2200, Accuracy: 0.862
max_features: 2300, Accuracy: 0.863
max_features: 2400, Accuracy: 0.865
max_features: 2500, Accuracy: 0.865
max_features: 2600, Accuracy: 0.871
max_features: 2700, Accuracy: 0.872
max_features: 2800, Accuracy: 0.873
max_

In [4]:
#띄어쓰기
spacing = Spacing()
spaced = spacing("아무래도 20년도지난작품이라 지금보기는너무유치하다")
print(spaced)

아무래도 20년도 지난 작품이라 지금 보기는 너무 유치하다


In [10]:
data.sample(n=5)

,id,document,label
3484,3485,정말 다시 보고 싶지만... 구할수 없어서 더욱더 안타까운 영화,1
2863,2864,쫌 오글거리지만 다시봐도 재밌는 드라마,1
2469,2470,전작보다 더 치밀해진 액션과 코믹요소들. 흥미진진한 스토리,1
2909,2910,도대체가 뭔 영화여? 뭘 말하려는거여? OO!!!,0
2418,2419,2014년 필수 영화를 꼽으라면 그 중 하나로 꼽을!,1
